In [1]:
!gdown 1dvWORIUL3zWi4Df4hNUeHYfzPM_YRRGs

Downloading...
From: https://drive.google.com/uc?id=1dvWORIUL3zWi4Df4hNUeHYfzPM_YRRGs
To: /home/yuuhanase/FPTU/EXE101/PaperClipAI_EnglishGrading/experiments/train.csv
100%|██████████████████████████████████████| 9.29M/9.29M [00:01<00:00, 8.01MB/s]


In [24]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


In [3]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files="train.csv")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text_id', 'full_text', 'cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions'],
        num_rows: 3911
    })
})

In [9]:
dataset['train']['full_text'][0]

"I think that students would benefit from learning at home,because they wont have to change and get up early in the morning to shower and do there hair. taking only classes helps them because at there house they'll be pay more attention. they will be comfortable at home.\n\nThe hardest part of school is getting ready. you wake up go brush your teeth and go to your closet and look at your cloths. after you think you picked a outfit u go look in the mirror and youll either not like it or you look and see a stain. Then you'll have to change. with the online classes you can wear anything and stay home and you wont need to stress about what to wear.\n\nmost students usually take showers before school. they either take it before they sleep or when they wake up. some students do both to smell good. that causes them do miss the bus and effects on there lesson time cause they come late to school. when u have online classes u wont need to miss lessons cause you can get everything set up and go t

In [25]:
tkn = tokenizer(dataset['train']['full_text'][0], return_tensors='pt')
tkn['input_ids'][0]
tokenizer.decode(tkn['input_ids'][0])

"[CLS] i think that students would benefit from learning at home, because they wont have to change and get up early in the morning to shower and do there hair. taking only classes helps them because at there house they'll be pay more attention. they will be comfortable at home. the hardest part of school is getting ready. you wake up go brush your teeth and go to your closet and look at your cloths. after you think you picked a outfit u go look in the mirror and youll either not like it or you look and see a stain. then you'll have to change. with the online classes you can wear anything and stay home and you wont need to stress about what to wear. most students usually take showers before school. they either take it before they sleep or when they wake up. some students do both to smell good. that causes them do miss the bus and effects on there lesson time cause they come late to school. when u have online classes u wont need to miss lessons cause you can get everything set up and go 

In [34]:
from unstructured.cleaners.core import clean_extra_whitespace

def clean_text(batch):
    text = batch['full_text']
    text = text.replace("\n", ' ')
    text = text.replace("\t", ' ')
    text = text.replace("\r", ' ')
    
    text = clean_extra_whitespace(text)
    batch['full_text'] = text
    return batch
    
clean_ds = dataset.map(clean_text)

Map:   0%|          | 0/3911 [00:00<?, ? examples/s]

In [36]:
def transform(batch):
    inputs = batch['full_text']
    targets_feat = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
    targets = []
    for feat in targets_feat:
        targets.append(batch[feat]/5)
    return {
        'text': inputs,
        'target': targets
    }

train_ds = clean_ds.map(transform)
train_ds

Map:   0%|          | 0/3911 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text_id', 'full_text', 'cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions', 'text', 'target'],
        num_rows: 3911
    })
})